In [ ]:
import pandas as pd
import numpy as np
from google.colab import files
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# data here is unique among each branch
# but might have duplicated product among all branches

file_name = '/content/drive/My Drive/Product_Classification/Data/data_output.csv'
data = pd.read_csv(file_name)
data.drop(['Unnamed: 0'], axis = 1, inplace = True)
print(data.shape)
print(data.columns)

(15354, 16)
Index(['ProductId', 'ProductName', 'Branch', 'level_1', 'level_2', 'level_3',
       'Image', 'ProductUrl', 'PricePerItem', 'PriceMode', 'HasMultiBuyDeal',
       'MultiBuyDeal', 'MultiBuyBasePrice', 'MultiBuyPrice',
       'MultiBuyQuantity', 'PromoBadgeImageLabel'],
      dtype='object')


# Data Preprocessing

## Variable information

In [ ]:
# 0. Delete feature since it is classified by promotion
category_set = list(set(data.level_1))
category_set.remove('Featured')
category_set.remove('Meal Spot')
data_unique = data[data.level_1.isin(category_set)]

In [ ]:
data_unique.shape

(14259, 16)

In [ ]:
# 1. Check whether there exists null values among variables
# It is possible that there exits null values for variables starting from MultiBuyDeal
# It is found that 'PromoBadgeImageLabel' is the variable representing pomotion
data_unique.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14259 entries, 26 to 15353
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductId             14259 non-null  object 
 1   ProductName           14259 non-null  object 
 2   Branch                14259 non-null  object 
 3   level_1               14259 non-null  object 
 4   level_2               14259 non-null  object 
 5   level_3               14259 non-null  object 
 6   Image                 14259 non-null  object 
 7   ProductUrl            14259 non-null  object 
 8   PricePerItem          14259 non-null  float64
 9   PriceMode             14259 non-null  object 
 10  HasMultiBuyDeal       14259 non-null  bool   
 11  MultiBuyDeal          795 non-null    object 
 12  MultiBuyBasePrice     14259 non-null  float64
 13  MultiBuyPrice         5552 non-null   float64
 14  MultiBuyQuantity      5552 non-null   float64
 15  PromoBadgeImageLab

In [ ]:
# 2. Check the relation between variables related to MultiBuy
# 2.1 Check relations among variables related to price
# we find that MultiBuyBasePrice = PricePerItem, PricePerItem != MultiBuyPrice
print(np.where(data_unique['MultiBuyBasePrice'] != data_unique['PricePerItem']))
print(np.where(data_unique['MultiBuyBasePrice'] != data_unique['MultiBuyPrice']))

(array([], dtype=int64),)
(array([    0,     1,     2, ..., 14255, 14256, 14257]),)


In [ ]:
# Summary of data 
# 1. The following variables exist data missing (which is allowable based on the definition): 
#    'MultiBuyDeal', 'MultiBuyPrice', 'MultiBuyQuantity', 'PromoBadgeImageLabel'
# 2. Promotion variables: 'PromoBadgeImageLabel' is Nan or non-empty
# 3. Duplicated variables: 'PricePerItem' = 'MultiBuyBasePrice'

## Nan Value and New Variables

In [ ]:
# We replace nan to meaning values for each variable, based on the variable definition

def replace(new_data_set, old_data_set):
  # 1. all 'object' nan into 'No'
  new_data_set['MultiBuyDeal'] = old_data_set['MultiBuyDeal'].fillna('No')
  new_data_set['PromoBadgeImageLabel'] = old_data_set['PromoBadgeImageLabel'].fillna('No')

  # 2. all 'quantitative' nan into 'float 0'
  new_data_set['MultiBuyPrice'] = old_data_set['MultiBuyPrice'].fillna(0)
  new_data_set['MultiBuyQuantity'] = old_data_set['MultiBuyQuantity'].fillna(0)

  print(new_data_set.info())


In [ ]:
# data_unique_within_branch: unique product within each branch
data_unique_within_branch = data_unique.copy()
replace(data_unique_within_branch, data_unique)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 14259 entries, 26 to 15353
Data columns (total 16 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   ProductId             14259 non-null  object 
 1   ProductName           14259 non-null  object 
 2   Branch                14259 non-null  object 
 3   level_1               14259 non-null  object 
 4   level_2               14259 non-null  object 
 5   level_3               14259 non-null  object 
 6   Image                 14259 non-null  object 
 7   ProductUrl            14259 non-null  object 
 8   PricePerItem          14259 non-null  float64
 9   PriceMode             14259 non-null  object 
 10  HasMultiBuyDeal       14259 non-null  bool   
 11  MultiBuyDeal          14259 non-null  object 
 12  MultiBuyBasePrice     14259 non-null  float64
 13  MultiBuyPrice         14259 non-null  float64
 14  MultiBuyQuantity      14259 non-null  float64
 15  PromoBadgeImageLab

## Cohort analysis



### Within each branch

In [ ]:
# 1. number of products for each branch
data_unique_within_branch.groupby(['Branch']).size()

Branch
New World Albany          227
New World Mt Roskill    14032
dtype: int64

In [ ]:
# 2. information of product price for each branch
data_unique_within_branch.groupby('Branch')['PricePerItem'].describe()

,count,mean,std,min,25%,50%,75%,max
Branch,,,,,,,,
New World Albany,227.0,6.022379,6.482522,1.79,3.49,4.99,6.99,78.99
New World Mt Roskill,14032.0,7.611759,7.263146,0.15,3.49,5.00,8.99,129.90


In [ ]:
data_unique_within_branch.groupby('Branch')['PricePerItem'].mean().sort_values(ascending=False).to_frame().reset_index()

,Branch,PricePerItem
0,New World Mt Roskill,7.611759
1,New World Albany,6.022379


In [ ]:
data_unique_within_branch.groupby(['Branch'])['PricePerItem'].std().sort_values(ascending=False).to_frame().reset_index()

,Branch,PricePerItem
0,New World Mt Roskill,7.263146
1,New World Albany,6.482522


In [ ]:
# 3. information of product number and price for each category of each branch
# Here we find the distributions of price vary among different categories,
# so it might be treated as a variable in further classification 
data_unique_within_branch.groupby(['Branch', 'level_1'])['PricePerItem'].describe()

count  ...     max
Branch               level_1                              ...        
New World Albany     Fresh Foods & Bakery          227.0  ...   78.99
New World Mt Roskill Baby, Toddler & Kids          354.0  ...   45.99
                     Beer, Cider & Wine           1236.0  ...  120.99
                     Chilled, Frozen & Desserts   1428.0  ...   42.99
                     Drinks                       1067.0  ...   25.99
                     Fresh Foods & Bakery         1723.0  ...  129.90
                     Kitchen, Dining & Household  1188.0  ...   99.99
                     Pantry                       4742.0  ...   35.99
                     Personal Care                1781.0  ...   54.99
                     Pets                          513.0  ...   37.49

[10 rows x 8 columns]

In [ ]:
data_unique_within_branch.groupby(['Branch', 'level_1'])['PricePerItem'].mean().sort_values(ascending=False).to_frame().reset_index()

,Branch,level_1,PricePerItem
0,New World Mt Roskill,"Beer, Cider & Wine",16.021561
1,New World Mt Roskill,Personal Care,9.992465
2,New World Mt Roskill,"Baby, Toddler & Kids",9.751017
3,New World Mt Roskill,Fresh Foods & Bakery,9.368114
4,New World Mt Roskill,Pets,7.862904
5,New World Mt Roskill,"Kitchen, Dining & Household",7.706776
6,New World Mt Roskill,"Chilled, Frozen & Desserts",6.320623
7,New World Albany,Fresh Foods & Bakery,6.022379
8,New World Mt Roskill,Drinks,5.531415
9,New World Mt Roskill,Pantry,4.533669


In [ ]:
data_unique_within_branch.groupby(['Branch', 'level_1'])['PricePerItem'].std().sort_values(ascending=False).to_frame().reset_index()

,Branch,level_1,PricePerItem
0,New World Mt Roskill,Fresh Foods & Bakery,10.143173
1,New World Mt Roskill,"Beer, Cider & Wine",9.880402
2,New World Mt Roskill,"Baby, Toddler & Kids",9.280134
3,New World Mt Roskill,Personal Care,7.434210
4,New World Mt Roskill,Pets,6.852083
5,New World Mt Roskill,"Kitchen, Dining & Household",6.848438
6,New World Albany,Fresh Foods & Bakery,6.482522
7,New World Mt Roskill,"Chilled, Frozen & Desserts",3.742553
8,New World Mt Roskill,Drinks,3.399076
9,New World Mt Roskill,Pantry,3.089238


### Among all branches

In [ ]:
# 1. Delete unique data among all branches
# check and delete duplicate products among all branches
# data_unique_all_branches: unique data among all branches
# data_unique_all_branches = data_unique_within_branch.drop_duplicates(subset=['ProductName'], keep='first')

data_unique_all_branches = data_unique_within_branch.drop_duplicates(subset=['ProductId'], keep='first')
print(data_unique_all_branches.shape)

(13261, 16)


In [ ]:
# 2. information of product number and price for each category
data_unique_all_branches.groupby(['level_1'])['PricePerItem'].describe()
# Use "price range" as a variable
# "Price range" is either mean +- 3std or quantile
# pros and cons between confidence interval and quantile (quantile: pros:: not affected by data cons: buzhidao quduoshao 5% 30%, Gaussian: pros: theoretical values, cons: must follow Gaussian distribution)

,count,mean,std,min,25%,50%,75%,max
level_1,,,,,,,,
"Baby, Toddler & Kids",354.0,9.751017,9.280134,0.99,2.015,6.99,14.9900,45.99
"Beer, Cider & Wine",1236.0,16.021561,9.880402,1.79,9.990,14.99,19.9900,120.99
"Chilled, Frozen & Desserts",1428.0,6.320623,3.742553,0.89,3.990,5.69,7.7900,42.99
Drinks,1067.0,5.531415,3.399076,0.89,3.490,4.69,6.9900,25.99
Fresh Foods & Bakery,1392.0,10.481157,11.170072,1.20,4.190,5.99,11.9225,129.90
"Kitchen, Dining & Household",1186.0,7.712875,6.852581,0.15,3.990,5.99,9.4975,99.99
Pantry,4310.0,4.348956,3.039659,0.50,2.490,3.49,4.9900,35.99
Personal Care,1775.0,10.014389,7.436933,0.90,4.790,7.89,12.9900,54.99
Pets,513.0,7.862904,6.852083,1.00,2.490,6.69,10.4900,37.49


In [ ]:
# 3. sort mean and std of product price for each category
data_unique_all_branches.groupby(['level_1'])['PricePerItem'].mean().sort_values(ascending=False).to_frame().reset_index()

,level_1,PricePerItem
0,"Beer, Cider & Wine",16.021561
1,Fresh Foods & Bakery,10.481157
2,Personal Care,10.014389
3,"Baby, Toddler & Kids",9.751017
4,Pets,7.862904
5,"Kitchen, Dining & Household",7.712875
6,"Chilled, Frozen & Desserts",6.320623
7,Drinks,5.531415
8,Pantry,4.348956


In [ ]:
data_unique_all_branches.groupby(['level_1'])['PricePerItem'].std().sort_values(ascending=False).to_frame().reset_index()

,level_1,PricePerItem
0,Fresh Foods & Bakery,11.170072
1,"Beer, Cider & Wine",9.880402
2,"Baby, Toddler & Kids",9.280134
3,Personal Care,7.436933
4,"Kitchen, Dining & Household",6.852581
5,Pets,6.852083
6,"Chilled, Frozen & Desserts",3.742553
7,Drinks,3.399076
8,Pantry,3.039659


In [ ]:
#@title Default title text
# 3. information of product for each promotion
data_unique_all_branches.groupby(['PromoBadgeImageLabel'])['PricePerItem', 'MultiBuyPrice', 'MultiBuyQuantity'].describe()

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: FutureWarning: Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.
  


PricePerItem                       ... MultiBuyQuantity          
                                      count       mean       std  ...              50%  75%  max
PromoBadgeImageLabel                                              ...                           
Club Deal                             921.0   6.124289  4.825669  ...              1.0  1.0  1.0
Everyday Value                        404.0   4.390025  4.591828  ...              0.0  0.0  0.0
Multibuy Store Saver                   21.0   3.232857  1.430235  ...              2.0  3.0  8.0
Multibuy Super Saver                  142.0   2.777394  0.777168  ...              2.0  2.0  5.0
Multibuy Super Saver Club Deal        127.0   2.716772  0.721160  ...              2.0  3.0  4.0
No                                   7628.0   8.377264  8.263497  ...              0.0  0.0  0.0
Saver                                3136.0   7.040724  6.043467  ...              1.0  1.0  1.0
Store Saver                           436.0  11.386170  8.312277  ...              1.0  1.0  1.0
Super Saver                           254.0   8.907008  7.274398  ...              1.0  1.0  1.0
Super Saver Club Deal                 192.0   5.433594  4.093292  ...              1.0  1.0  1.0

[10 rows x 24 columns]

## Word frequency

In [ ]:
# word frequency of product name
word_freq = data_unique_all_branches['ProductName'].str.split(expand = True).stack().value_counts().to_frame().reset_index()
word_freq.rename(columns = {'index':'Word', 0:'Word Frequency'}, inplace = True)
word_freq['Percent'] = word_freq['Word Frequency'] / word_freq['Word Frequency'].sum()
word_freq.head(15)

,Word,Word Frequency,Percent
0,&,1839,0.024426
1,Pams,850,0.011290
2,Chicken,520,0.006907
3,Chocolate,493,0.006548
4,Cheese,412,0.005472
5,Organic,382,0.005074
6,Fresh,370,0.004914
7,Sauce,364,0.004835
8,Food,361,0.004795
9,Cream,360,0.004782


In [ ]:
# word frequency (number and percemt) of product name by category
word_cat_table = data_unique_all_branches[['level_1','ProductName']]
word_cat_dict = {k: v["ProductName"].tolist() for k,v in word_cat_table.groupby('level_1')}

# word set for further use
word_freq_cat_list = []

for key, name_list in word_cat_dict.items():

  word_freq_by_cat = {}

  for name in name_list:

    for word in name.split():
      if word not in word_freq_by_cat:
        word_freq_by_cat[word] = 1
      else:
        word_freq_by_cat[word] += 1
    
  word_freq_cat_list.append({
      key: word_freq_by_cat
  })



In [ ]:
# word frequency table by category

rows = []

for data in word_freq_cat_list:

  category = list(data.keys())[0]
  
  for _, values in data.items():

    word_list = list(values.keys())
    freq_list = list(values.values())

    for i in range(len(word_list)):
      row_dict = {}
      row_dict['Category'] = category
      row_dict['Word'] = word_list[i]
      row_dict['Word Frequency'] = freq_list[i]
    
      rows.append(row_dict)


In [ ]:
# From the frequency table, it is found frequencies of certain words will be high in every category

# 1. percentage: # word_freq_cat['Percent'] = word_freq_cat['Word Frequency'] / word_freq_cat['Word Frequency'].loc[word_freq_cat['Category']].sum()
# 2. percentage of frequency in products
word_freq_cat = pd.DataFrame(rows)
word_freq_cat.sort_values(['Category', 'Word Frequency'], ascending = False, inplace = True)
word_freq_cat = word_freq_cat[word_freq_cat['Word'] != '&']
word_freq_cat

,Category,Word,Word Frequency
10652,Pets,Food,306
10644,Pets,Cat,242
10642,Pets,Dog,149
10682,Pets,Chicken,132
10677,Pets,Purina,119
...,...,...,...
466,"Baby, Toddler & Kids",Sensitive,1
467,"Baby, Toddler & Kids",Sprout,1
468,"Baby, Toddler & Kids",Lentils,1
469,"Baby, Toddler & Kids",Beans,1


In [ ]:
# Top 5 frequent word for each category
word_frequency_top_5_cat = word_freq_cat.sort_values(by=["Category","Word Frequency"], ascending=[True, False]).groupby("Category").head(1).reset_index()
word_frequency_top_5_cat.drop('index', axis = 1)

,Category,Word,Word Frequency
0,"Baby, Toddler & Kids",Months,131
1,"Beer, Cider & Wine",Pinot,183
2,"Chilled, Frozen & Desserts",Cheese,298
3,Drinks,Drink,265
4,Fresh Foods & Bakery,Produce,164
5,"Kitchen, Dining & Household",Laundry,102
6,Pantry,Pams,425
7,Personal Care,Hair,146
8,Pets,Food,306


In [ ]:
# dowload data_unique_all_branches
data_unique_all_branches.to_csv('data_unqiue_all_branches.csv')